# Prétraitement des Données Textuelles pour la Prédiction de Sentiment

## Contexte

Ce notebook couvre la phase de prétraitement des données textuelles pour notre projet d'analyse de sentiment. Suite à l'analyse exploratoire des données (EDA) réalisée dans le notebook `1_data_exploration.ipynb`, nous savons que les données sont relativement propres (pas de valeurs manquantes majeures) mais nécessitent une transformation pour être utilisables par les modèles de machine learning.

## Objectifs du Prétraitement

Les principaux objectifs de cette phase sont :
1. Nettoyer le texte des commentaires (supprimer le bruit, normaliser la casse).
2. Transformer le texte en une représentation numérique (vectorisation).
3. Préparer le dataset pour l'entraînement du modèle en le divisant en ensembles d'entraînement et de test.
4. Prendre en compte les observations de l'EDA, notamment la distribution des sentiments.

## 2. Importation des Bibliothèques et Chargement des Données

Nous importons les bibliothèques nécessaires et chargeons le dataset nettoyé si des modifications ont été faites lors de l'EDA (sinon, le dataset original).

In [1]:
# Importation des bibliothèques
import pandas as pd
import numpy as np
import re # Pour les expressions régulières (nettoyage de texte)
import nltk # Pour le traitement du langage naturel (tokenisation, stopwords, etc.)
import string # Pour la ponctuation
from nltk.corpus import stopwords # Pour les mots vides
from nltk.stem import PorterStemmer, WordNetLemmatizer # Pour le stemming et la lemmatisation
from sklearn.model_selection import train_test_split # Pour diviser les données
from sklearn.feature_extraction.text import TfidfVectorizer # Pour la vectorisation TF-IDF

try:
    stop_words_english = set(stopwords.words('english'))
    # nltk.data.find('corpora/wordnet') # Vérifie si WordNet est installé (pour lemmatisation)
except LookupError:
    print("Téléchargement des ressources NLTK (stopwords, wordnet)...")
    nltk.download('stopwords')
    nltk.download('wordnet')
    nltk.download('omw-1.4') # Open Multilingual WordNet (souvent nécessaire pour WordNet)
    stop_words_english = set(stopwords.words('english'))
    print("Téléchargement terminé.")


# Chargement du dataset
file_path = '../data/sentiment_analysis.csv'
text_column = 'text'
sentiment_column = 'sentiment'

try:
    df = pd.read_csv(file_path)
    print("Dataset chargé avec succès.")
    print(f"Dimensions du dataset : {df.shape}")

except FileNotFoundError:
    print(f"Erreur : Le fichier {file_path} n'a pas été trouvé. Placez-le dans le dossier 'data/'.")
    df = None

Dataset chargé avec succès.
Dimensions du dataset : (499, 7)


## 3. Nettoyage et Normalisation du Texte

Cette section applique une série d'opérations pour nettoyer les commentaires et les préparer à la vectorisation.

In [2]:
# Fonction pour nettoyer un commentaire
def clean_text(text):
    if isinstance(text, str): # S'assurer que l'entrée est une chaîne
        text = text.lower() # 1. Convertir en minuscules
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # 2. Supprimer les URLs
        text = re.sub(r'\@\w+|\#','', text) # 3. Supprimer les mentions (@) et hashtags (#)
        text = re.sub(r'\d+', '', text) # 4. Supprimer les chiffres
        text = text.translate(str.maketrans('', '', string.punctuation)) # 5. Supprimer la ponctuation
        text = text.strip() # 6. Supprimer les espaces blancs en début et fin
        # Suppression des espaces multiples
        text = re.sub(r'\s+', ' ', text).strip()

        return text
    else:
        return "" # Retourner une chaîne vide

In [9]:
if df is not None and text_column in df.columns:
    print(f"Nettoyage de la colonne '{text_column}'...")
    df['cleaned_text'] = df[text_column].apply(clean_text)
    print("Nettoyage terminé.")

    # Afficher quelques exemples après nettoyage
    print("\nExemples de commentaires après nettoyage :")
    display(df[[text_column, 'cleaned_text']].head())
elif df is not None:
    print(f"Impossible d'appliquer le nettoyage : la colonne de texte '{text_column}' n'a pas été trouvée.")

Nettoyage de la colonne 'text'...
Nettoyage terminé.

Exemples de commentaires après nettoyage :


,text,cleaned_text
0,What a great day!!! Looks like dream.,what a great day looks like dream
1,"I feel sorry, I miss you here in the sea beach",i feel sorry i miss you here in the sea beach
2,Don't angry me,dont angry me
3,We attend in the class just for listening teac...,we attend in the class just for listening teac...
4,"Those who want to go, let them go",those who want to go let them go


### Tokenisation, Suppression des Mots Vides et Normalisation

Après le nettoyage de base, nous allons :
1. **Tokeniser** le texte (diviser en mots).
2. Supprimer les **mots vides** (stop words) qui n'apportent pas de sens pour la classification de sentiment.
3. Appliquer une **normalisation** pour réduire les mots à leur forme de base (stemming ou lemmatisation).

Nous allons utiliser la **Lemmatisation** car elle est généralement plus précise que le stemming (elle réduit les mots à leur forme canonique en tenant compte du contexte et du vocabulaire, plutôt qu'une simple règle de troncature).

In [10]:
# Initialisation du Lemmatizer (pour WordNet)
lemmatizer = WordNetLemmatizer()

def advanced_text_processing(text):
    if isinstance(text, str):
        # Tokenisation
        tokens = text.split() # Tokenisation simple par espace après nettoyage de la ponctuation

        # Suppression des mots vides et lemmatisation
        processed_tokens = [
            lemmatizer.lemmatize(word) for word in tokens
            if word not in stop_words_english and len(word) > 1 # Supprime stopwords et mots courts
        ]

        return " ".join(processed_tokens)
    return ""

# La fonction `advanced_text_processing` prend le texte déjà nettoyé. Elle le divise en mots (`split()`), filtre les mots vides
# (`stop_words_english`) et les mots courts, puis applique la lemmatisation (`lemmatizer.lemmatize()`) pour réduire les mots à 
# leur forme de base. Le résultat est une chaîne de texte prétraitée prête pour la vectorisation.

In [ ]:
if df is not None and 'cleaned_text' in df.columns:
    print(f"Application du traitement avancé (tokenisation, stopwords, lemmatisation) sur 'cleaned_text'...")
    # Appliquez la fonction sur la colonne de texte nettoyée
    df['processed_text'] = df['cleaned_text'].apply(advanced_text_processing)
    print("Traitement avancé terminé.")

    # Afficher quelques exemples après traitement avancé
    print("\nExemples de commentaires après traitement avancé :")
    display(df[['cleaned_text', 'processed_text']].head())

    df = df.drop('cleaned_text', axis=1) # Supprimez la colonne nettoyée pour économiser de l'espace mémoire
    print("Colonne 'cleaned_text' supprimée.")

elif df is not None:
     print(f"Erreur : La colonne 'cleaned_text' n'a pas été trouvée. Assurez-vous que le nettoyage a bien été exécuté.")
     
     
#  Nous créons une nouvelle colonne `processed_text` contenant les commentaires après tokenisation,
#  suppression des mots vides et lemmatisation. L'affichage des exemples permet de vérifier le résultat. 
#  À ce stade, la colonne `processed_text` est prête pour la vectorisation.


Application du traitement avancé (tokenisation, stopwords, lemmatisation) sur 'cleaned_text'...
Traitement avancé terminé.

Exemples de commentaires après traitement avancé :


,cleaned_text,processed_text
0,what a great day looks like dream,great day look like dream
1,i feel sorry i miss you here in the sea beach,feel sorry miss sea beach
2,dont angry me,dont angry
3,we attend in the class just for listening teac...,attend class listening teacher reading slide n...
4,those who want to go let them go,want go let go


Colonne 'cleaned_text' supprimée.


## 4. Vectorisation du Texte (TF-IDF)

Nous allons convertir le texte prétraité (`processed_text`) en une représentation numérique. Nous utiliserons la technique TF-IDF (`TfidfVectorizer` de scikit-learn). TF-IDF pondère les mots en fonction de leur fréquence dans un document et de leur rareté dans l'ensemble du corpus.

In [12]:
if df is not None and 'processed_text' in df.columns and sentiment_column in df.columns:
    print("Vectorisation du texte en utilisant TF-IDF...")

    # Initialisation du TfidfVectorizer
    # min_df: ignore les termes qui apparaissent dans moins de X documents
    # max_df: ignore les termes qui apparaissent dans plus de X% des documents
    # ngram_range: considère des n-grammes de 1 mot (unigrammes)
    tfidf_vectorizer = TfidfVectorizer(max_features=5000, # Limite le vocabulaire aux 5000 mots/n-grammes les plus fréquents
                                       min_df=5,         # Ignore les termes qui apparaissent dans moins de 5 documents
                                       max_df=0.8)       # Ignore les termes qui apparaissent dans plus de 80% des documents

    # Adaptez le vectoriseur sur le texte prétraité et transformez-le
    X = tfidf_vectorizer.fit_transform(df['processed_text'])

    # La variable cible (sentiment)
    y = df[sentiment_column]

    print("Vectorisation terminée.")
    print(f"Forme de la matrice TF-IDF (nombre de documents, taille du vocabulaire) : {X.shape}")
    print(f"Forme du vecteur cible : {y.shape}")

    # Afficher les noms des features (mots) dans l'ordre
    print("\nQuelques exemples de features (mots) dans le vocabulaire TF-IDF :")
    display(tfidf_vectorizer.get_feature_names_out()[:30])

elif df is not None:
     print(f"Erreur : Les colonnes 'processed_text' ou '{sentiment_column}' n'ont pas été trouvées. Vérifiez les étapes précédentes.")
else:
     print("Erreur : Le DataFrame n'est pas chargé.")

Vectorisation du texte en utilisant TF-IDF...
Vectorisation terminée.
Forme de la matrice TF-IDF (nombre de documents, taille du vocabulaire) : (499, 130)
Forme du vecteur cible : (499,)

Quelques exemples de features (mots) dans le vocabulaire TF-IDF :


array(['actually', 'always', 'amazing', 'baby', 'back', 'bad', 'best',
       'better', 'big', 'bit', 'cant', 'car', 'coffee', 'come', 'coming',
       'could', 'day', 'didnt', 'done', 'dont', 'dude', 'ever',
       'everyone', 'family', 'feel', 'feeling', 'finally', 'find',
       'finished', 'first'], dtype=object)

## 5. Division des Données (Entraînement/Test)

Nous allons diviser le dataset vectorisé en un ensemble d'entraînement et un ensemble de test. L'ensemble d'entraînement sera utilisé pour entraîner le modèle, et l'ensemble de test pour évaluer ses performances sur des données invisibles.

Une division typique est 80% pour l'entraînement et 20% pour le test. Nous allons également stratifier la division pour nous assurer que la proportion de chaque classe de sentiment est la même dans les ensembles d'entraînement et de test, ce qui est important étant donné le léger déséquilibre des classes.

In [13]:
if 'X' in locals() and 'y' in locals():
    print("Division des données en ensembles d'entraînement et de test...")

    # Division en entraînement (80%) et test (20%)
    # random_state: assure la reproductibilité de la division
    # stratify=y: assure que les proportions des classes de sentiment sont maintenues dans les deux ensembles
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


    print("Division terminée.")
    print(f"Forme de l'ensemble d'entraînement (X_train) : {X_train.shape}")
    print(f"Forme de l'ensemble de test (X_test) : {X_test.shape}")
    print(f"Forme des labels d'entraînement (y_train) : {y_train.shape}")
    print(f"Forme des labels de test (y_test) : {y_test.shape}")

    # Vérifier la distribution des classes dans les ensembles
    print("\nDistribution des sentiments dans y_train :")
    print(y_train.value_counts(normalize=True) * 100)
    print("\nDistribution des sentiments dans y_test :")
    print(y_test.value_counts(normalize=True) * 100)

else:
     print("Erreur : Les variables X ou y n'ont pas été créées. Vérifiez l'étape de vectorisation.")



Division des données en ensembles d'entraînement et de test...
Division terminée.
Forme de l'ensemble d'entraînement (X_train) : (399, 130)
Forme de l'ensemble de test (X_test) : (100, 130)
Forme des labels d'entraînement (y_train) : (399,)
Forme des labels de test (y_test) : (100,)

Distribution des sentiments dans y_train :
sentiment
neutral     39.849624
positive    33.333333
negative    26.817043
Name: proportion, dtype: float64

Distribution des sentiments dans y_test :
sentiment
neutral     40.0
positive    33.0
negative    27.0
Name: proportion, dtype: float64


*Commentaire :* Nous utilisons `train_test_split` pour diviser les données. `test_size=0.2` alloue 20% des données au test. `random_state=42` assure que la division est la même à chaque exécution. L'argument `stratify=y` est très important ici car il maintient les proportions des classes de sentiment dans les deux ensembles, ce qui évite d'avoir un ensemble de test avec très peu d'exemples des classes minoritaires. L'affichage des formes et de la distribution des classes vérifie que la division s'est bien passée.

## 6. Conclusion du Prétraitement et Prochaines Étapes

Dans ce notebook, nous avons appliqué une série d'étapes pour prétraiter les commentaires textuels :
- Nettoyage de base (minuscules, suppression de ponctuation, URLs, chiffres, etc.).
- Tokenisation, suppression des mots vides et lemmatisation.
- Transformation du texte prétraité en vecteurs numériques à l'aide de TF-IDF.
- Division du dataset vectorisé en ensembles d'entraînement et de test stratifiés.

Le résultat de cette phase est :
- `X_train` et `y_train` : Les données d'entraînement (features vectorisées et labels).
- `X_test` et `y_test` : Les données de test (features vectorisées et labels).
- Le `tfidf_vectorizer` entraîné, qui sera nécessaire pour prétraiter de nouveaux commentaires pour la prédiction.

Ces ensembles sont maintenant prêts à être utilisés pour entraîner différents modèles de machine learning pour la classification de sentiment.

### Prochaines Étapes :

La phase suivante consistera à :
1. Sélectionner et entraîner plusieurs modèles de classification (par exemple, Naïve Bayes, Régression Logistique, SVM, potentiellement des modèles de deep learning).
2. Évaluer les performances de chaque modèle sur l'ensemble de test en utilisant des métriques appropriées (Accuracy, Precision, Recall, F1-score, Matrice de confusion), en portant une attention particulière aux classes minoritaires en raison du léger déséquilibre.
3. Comparer les modèles et sélectionner le "meilleur" modèle en justifiant ce choix par les performances et les contraintes d'intégration web (vitesse d'inférence, taille du modèle).